In [2]:
import json
import pandas as pd
import requests

popular = []
for i in range(1,1000):
    url = 'https://api.themoviedb.org/3/movie/popular?api_key={1}&language=ko-KR&page={0}'.format(i,'36927ad1d2817ff0fa31947e47d186fd')
    resp = requests.get(url)
    popular.append(json.loads(resp.content))

    if i == 10:
        break

popular_movies = []
for row in popular:
    for result in row ['results']:
        popular_movies.append(result)

In [3]:
popular_movies=pd.DataFrame(popular_movies)
popular_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              200 non-null    bool   
 1   backdrop_path      199 non-null    object 
 2   genre_ids          200 non-null    object 
 3   id                 200 non-null    int64  
 4   original_language  200 non-null    object 
 5   original_title     200 non-null    object 
 6   overview           200 non-null    object 
 7   popularity         200 non-null    float64
 8   poster_path        200 non-null    object 
 9   release_date       200 non-null    object 
 10  title              200 non-null    object 
 11  video              200 non-null    bool   
 12  vote_average       200 non-null    float64
 13  vote_count         200 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 19.3+ KB


In [4]:
# popular_movies.to_csv("../data-files/popular_movies.csv", header=True, index=True)

In [4]:
popular_movies=popular_movies.drop(['adult', 'backdrop_path', 'video','genre_ids'], axis=1)

In [5]:
popular_movies.to_csv("../data-files/popular_movies.csv", header=True, index=True)
popular_movies.head(2)

,id,original_language,original_title,overview,popularity,poster_path,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,8986.935,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,4134
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,3395.495,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.8,1812


In [5]:
# 칼럼이름을 DB 칼럼이름과 동일하게 변경

popular_movies=popular_movies.rename(columns={'id':'movie_id', 'original_title':'subtitle','poster_path':'posterpath'})
popular_movies.head(2)

,movie_id,original_language,subtitle,overview,popularity,posterpath,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,8986.935,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,4134
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,3395.495,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.8,1812


In [6]:
popular_movies.isna().sum()

movie_id             0
original_language    0
subtitle             0
overview             0
popularity           0
posterpath           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [7]:
popular_movies_ids=popular_movies[["movie_id"]]
popular_movies_ids

,movie_id
0,580489
1,512195
2,585245
3,566525
4,589761
...,...
195,532067
196,522478
197,8960
198,81774


In [14]:
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

# cursor.execute('DELETE FROM popular_movies')
error_list=[]
for movie_id in popular_movies_ids.values:
    # print(type(movie_id), movie_id)  # movie_id type이 nd.arrary임.
    try:
        sql= "INSERT INTO popular_movies VALUES(%s)"
        cursor.execute(sql, movie_id[0]) 
    except:
        error_list.append(movie_id)
    
conn.commit()
cursor.close()
conn.close()

In [15]:
len(error_list)

185

In [11]:
#없는 영화 업데이트

popular_movies = []
for i in error_list:
        for e in i:
                url="https://api.themoviedb.org/3/movie/{0}?api_key={1}&language=ko-KR".format(e,'36927ad1d2817ff0fa31947e47d186fd')
                response = requests.get(url)
                popular_movies.append(json.loads(response.content))

In [12]:
movies2 = pd.DataFrame(popular_movies)
movies2

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/1zgob2Z8xVE3RZUgOKnHKcVPzOE.jpg,None,0,"[{'id': 16, 'name': '애니메이션'}, {'id': 35, 'name...",https://www.disneyplus.com/movies/the-simpsons...,877183,tt15829632,en,The Simpsons in Plusaversary,...,2021-11-12,0,4,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,더 심슨 플러스 기념일,False,6.8,143
1,False,/qXctHIfK4LAgHH7qZiJJVCPclaY.jpg,None,45223,"[{'id': 16, 'name': '애니메이션'}, {'id': 35, 'name...",https://www.disneyplus.com/movies/ciao-alberto...,876716,tt15476620,en,Ciao Alberto,...,2021-11-12,0,7,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,챠오 알베르토,False,7.6,239
2,False,/ng6SSB3JhbcpKTwbPDsRwUYK8Cq.jpg,"{'id': 531241, 'name': '스파이더맨(MCU) 시리즈', 'post...",160000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...",https://www.marvel.com/movies/spider-man-far-f...,429617,tt6320628,en,Spider-Man: Far From Home,...,2019-06-28,1131927996,129,"[{'english_name': 'Czech', 'iso_639_1': 'cs', ...",Released,모든 것이 다시 시작된다,스파이더맨: 파 프롬 홈,False,7.5,11134
3,False,/pFnEAK3eN8xX9AeggG9ISFYr1OO.jpg,None,0,"[{'id': 53, 'name': '스릴러'}]",,835551,tt11448490,en,Masquerade,...,2021-07-30,0,80,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Masquerade,False,5.9,29
4,False,/iWrDibFusV4JEz5Qw79RuWSGfX4.jpg,"{'id': 125574, 'name': '어메이징 스파이더맨 시리즈', 'post...",200000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...",http://www.theamazingspiderman.com,102382,tt1872181,en,The Amazing Spider-Man 2,...,2014-04-16,708962323,142,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,진짜 전쟁은 지금부터다,어메이징 스파이더맨 2,False,6.4,10007
5,False,/5PQhNs4M3RxIqaNAdBI6mXlXyYh.jpg,None,0,"[{'id': 27, 'name': '공포'}, {'id': 14, 'name': ...",,839436,tt13563520,en,Dragon Fury,...,2021-06-15,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Dragon Fury,False,6.4,28
6,False,/daOd8KBGDdsOhVdzXB3KU5fgBd2.jpg,None,0,"[{'id': 18, 'name': '드라마'}]",,453354,tt6194530,fr,Ava,...,2017-06-21,0,106,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,,아바,False,6.0,122
7,False,/q4h2ZbhHKhnWvrrqzcjT778zlbn.jpg,None,0,"[{'id': 16, 'name': '애니메이션'}, {'id': 878, 'nam...",,831405,tt5012504,en,Injustice,...,2021-10-09,0,78,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,인저스티스,False,8.0,391
8,False,/ux6gkGSKNFtp2NFYxwYFxVWdnGa.jpg,None,0,"[{'id': 80, 'name': '범죄'}, {'id': 18, 'name': ...",https://www.netflix.com/title/81346326,874948,tt13717980,tr,Kin,...,2021-10-08,0,106,"[{'english_name': 'Turkish', 'iso_639_1': 'tr'...",Released,영화 '악의 연대기' 터키 리메이크 작품,킨,False,6.6,32
9,False,/srYya1ZlI97Au4jUYAktDe3avyA.jpg,"{'id': 468552, 'name': '원더 우먼 시리즈', 'poster_pa...",200000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...",https://www.warnerbros.com/movies/wonder-woman...,464052,tt7126948,en,Wonder Woman 1984,...,2020-12-16,165160005,150,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"세상이 기다린 히어로, 희망이 되다",원더 우먼 1984,False,6.6,6224


In [13]:
# 3. DB에 저장 ( "resource you requested could not be found."행 영화는 제거하고 필요한 컬럼만 가져온 후 저장)
import numpy as np
# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# cursor.execute("delete from movie") #기존 데이터를 지울 때 주석을 풀어주세요 

error_list = []   ##결과 18개
            
# movies2['original_language'] = movies2['original_language'].apply(lambda ol: ol if ol else '')
movies2['overview'] = movies2['overview'].apply(lambda ov : ov if str(ov) != 'nan' else '')
movies2['release_date'] = movies2['release_date'].apply(lambda ov : ov if str(ov) != 'nan' else '')
movies2['poster_path'] = movies2['poster_path'].apply(lambda ov : ov if str(ov) != 'nan' else '')
print(movies2.values.shape)
##결과: 630186 entries  ???????????????

# 3. execute command
sql = """insert into movie (movie_id, title, subtitle, original_language, overview, popularity, release_date, vote_average, vote_count, posterpath) 
        values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
for movie in movies2.values:
    cursor.execute(sql, list(movie[list(np.array([9, 24, 12, 11, 13, 14, 18, 26, 27, 15]) - 3)]))

# movie3.append(list(movies2.values)) ##결과: 467108 entries    ????????????????????
                

conn.commit() # confirm previous execution
# 4. close resource
cursor.close()
conn.close()

(15, 25)
